#Pull Data from PGAdmin

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-14 17:42:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2021-02-14 17:42:17 (6.04 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles

In [5]:
#import library psycopyg2
import psycopg2

#import library pandas
import pandas as pd

#import library sqlio
import pandas.io.sql as sqlio

#create database connection variable 
conn = psycopg2.connect(user="postgres", password="ilovegroup3", host="finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com", database="housingdata")

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
#define query
query = "select * from census_housing_data"
#define query
query1 = "select * from crime_data"
#define query
query2 = "select * from coord_data"
#define query
query3 = "select * from city_data"

In [7]:
#execute query and save it to a variable
census_housing_data = sqlio.read_sql_query(query,conn)
crime_data = sqlio.read_sql_query(query1,conn)
coord_data = sqlio.read_sql_query(query2,conn)
city_data = sqlio.read_sql_query(query3,conn)

#Joining city_data and census_housing_data

In [8]:
# Setting zipcode column as index so as more easily join the table
city_data = city_data.set_index('zipcode')
census_housing_data = census_housing_data.set_index('zipcode')

In [ ]:
city_census_data = census_housing_data.join(city_data, how="left")

In [12]:
# Drop null values from city_census_data
city_census_data = city_census_data.dropna()

# Reset index for new city_census_data table
city_census_data = city_census_data.reset_index()
city_census_data

,zipcode,med_income,zhvi,total_pop,male,female,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,city
0,61531.0,56528.0,84009.17,3335.0,1647.0,1688.0,40.9,3274.0,0.0,38.0,77.0,0.0,8.0,77.0,3258.0,Farmington
1,61532.0,51833.0,46023.33,645.0,322.0,323.0,36.7,644.0,0.0,0.0,0.0,0.0,1.0,0.0,645.0,Forest City
2,61533.0,60250.0,117554.17,2509.0,1274.0,1235.0,41.9,2489.0,0.0,0.0,0.0,0.0,20.0,27.0,2482.0,Glasford
3,61534.0,57917.0,111049.92,1806.0,946.0,860.0,42.4,1796.0,9.0,3.0,0.0,0.0,0.0,25.0,1781.0,Green Valley
4,61535.0,81964.0,180972.58,1697.0,966.0,731.0,41.6,1662.0,16.0,41.0,0.0,0.0,0.0,0.0,1697.0,Groveland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,61525.0,122000.0,244723.75,9188.0,4553.0,4635.0,36.2,7211.0,306.0,47.0,1670.0,0.0,235.0,306.0,8882.0,Dunlap
1156,61526.0,81875.0,184235.33,810.0,315.0,495.0,47.6,798.0,12.0,4.0,0.0,0.0,0.0,13.0,797.0,Edelstein
1157,61528.0,105918.0,239963.17,3127.0,1647.0,1480.0,39.7,2778.0,0.0,0.0,341.0,0.0,21.0,244.0,2883.0,Edwards
1158,61529.0,70750.0,117759.25,2699.0,1319.0,1380.0,43.4,2653.0,0.0,0.0,60.0,0.0,0.0,31.0,2668.0,Elmwood


#Aggregate data by zipcode into data by city

In [13]:
#Create a dictionary to encompass data points that will be summed when aggregation by city occurs
sums = ["total_pop", "male", "female", "race_w", "race_aa", "race_nat", "race_as", "race_api", "race_oth", "eth_hisp", "eth_nonhisp"]
agg_dict = {sum:"mean" for sum in sums}

# Update dictionary agg_dict to include data points that will need to be averaged when aggregation by city occurs
agg_dict.update({"med_income":"mean", "med_age":"mean","zhvi":"mean"})

In [14]:
#Group data by city and apply agg_dict
city_census_data = city_census_data.groupby(["city"]).agg(agg_dict)

#Drop any potential null values
city_census_data.dropna()
city_census_data

,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_income,med_age,zhvi
city,,,,,,,,,,,,,,
Abingdon,3892.0,1731.0,2161.0,3865.0,55.0,62.0,0.0,0.0,0.0,39.0,3853.0,44570.0,39.4,59676.67
Adair,444.0,274.0,170.0,444.0,0.0,0.0,0.0,0.0,0.0,12.0,432.0,68594.0,54.6,62941.75
Addison,39216.0,19281.0,19935.0,29293.0,1812.0,401.0,3905.0,38.0,4986.0,15603.0,23613.0,64915.0,36.7,263669.50
Albany,1121.0,531.0,590.0,1119.0,0.0,2.0,0.0,0.0,2.0,47.0,1074.0,55000.0,45.4,117470.75
Albion,3351.0,1641.0,1710.0,3262.0,45.0,34.0,25.0,14.0,14.0,67.0,3284.0,49831.0,43.4,55956.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yale,430.0,200.0,230.0,409.0,18.0,70.0,0.0,0.0,0.0,0.0,430.0,55972.0,44.8,56292.50
Yates City,1191.0,589.0,602.0,1186.0,32.0,20.0,0.0,0.0,0.0,0.0,1191.0,56607.0,45.2,87263.75
Yorkville,24886.0,12460.0,12426.0,21503.0,1883.0,120.0,849.0,0.0,1193.0,3049.0,21837.0,95878.0,35.5,253435.08


# Merge crime data and city_census_data

In [15]:
# Set city as the index for crime_data to more easily merge with city_census_data
crime_data = crime_data.set_index('city')

In [16]:
# Join city_census_data and crime_data into final table
final_table = city_census_data.join(crime_data, how="left")

# Replace any null values with 0
# This will only impact null values from the crime data as any null values from other columns had previously been removed
final_table = final_table.fillna(0)
final_table

,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_income,med_age,zhvi,vio_crime,murder_mansl,rape,robbery,agg_assault,prop_crime,burglary,lar_theft,car_theft,arson
city,,,,,,,,,,,,,,,,,,,,,,,,
Abingdon,3892.0,1731.0,2161.0,3865.0,55.0,62.0,0.0,0.0,0.0,39.0,3853.0,44570.0,39.4,59676.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Adair,444.0,274.0,170.0,444.0,0.0,0.0,0.0,0.0,0.0,12.0,432.0,68594.0,54.6,62941.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Addison,39216.0,19281.0,19935.0,29293.0,1812.0,401.0,3905.0,38.0,4986.0,15603.0,23613.0,64915.0,36.7,263669.50,61.0,1.0,18.0,6.0,36.0,464.0,65.0,361.0,38.0,4.0
Albany,1121.0,531.0,590.0,1119.0,0.0,2.0,0.0,0.0,2.0,47.0,1074.0,55000.0,45.4,117470.75,1.0,0.0,1.0,0.0,0.0,6.0,2.0,4.0,0.0,0.0
Albion,3351.0,1641.0,1710.0,3262.0,45.0,34.0,25.0,14.0,14.0,67.0,3284.0,49831.0,43.4,55956.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yale,430.0,200.0,230.0,409.0,18.0,70.0,0.0,0.0,0.0,0.0,430.0,55972.0,44.8,56292.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yates City,1191.0,589.0,602.0,1186.0,32.0,20.0,0.0,0.0,0.0,0.0,1191.0,56607.0,45.2,87263.75,1.0,0.0,1.0,0.0,0.0,8.0,1.0,7.0,0.0,0.0
Yorkville,24886.0,12460.0,12426.0,21503.0,1883.0,120.0,849.0,0.0,1193.0,3049.0,21837.0,95878.0,35.5,253435.08,18.0,0.0,8.0,3.0,7.0,200.0,17.0,178.0,5.0,2.0


#Merge coord_data with final_table

In [ ]:
# Set coord_data index as city
coord_data = coord_data.set_index('city')

In [19]:
#Join final table with coord_data
final_table = final_table.join(coord_data, how="left")

In [20]:
# Drop null values from lat and long 
final_table = final_table.dropna(subset=['lat', 'long'])

In [21]:
# Drop vio_crime, prop_crime, and county_name columns as they will not be used in the final analysis
final_table = final_table.drop(["vio_crime","prop_crime","county_name"], axis = 1)

# Reset index 
final_table = final_table.reset_index()

In [22]:
# Drop any duplicate cities
final_table = final_table.drop_duplicates(subset=['city'])
final_table

,city,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_income,med_age,zhvi,murder_mansl,rape,robbery,agg_assault,burglary,lar_theft,car_theft,arson,lat,long
0,Abingdon,3892.0,1731.0,2161.0,3865.0,55.0,62.0,0.0,0.0,0.0,39.0,3853.0,44570.0,39.4,59676.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.804483,-90.401795
1,Adair,444.0,274.0,170.0,444.0,0.0,0.0,0.0,0.0,0.0,12.0,432.0,68594.0,54.6,62941.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.418931,-90.496794
2,Addison,39216.0,19281.0,19935.0,29293.0,1812.0,401.0,3905.0,38.0,4986.0,15603.0,23613.0,64915.0,36.7,263669.50,1.0,18.0,6.0,36.0,65.0,361.0,38.0,4.0,41.931696,-87.988956
3,Albany,1121.0,531.0,590.0,1119.0,0.0,2.0,0.0,0.0,2.0,47.0,1074.0,55000.0,45.4,117470.75,0.0,1.0,0.0,0.0,2.0,4.0,0.0,0.0,41.789475,-90.219294
4,Albion,3351.0,1641.0,1710.0,3262.0,45.0,34.0,25.0,14.0,14.0,67.0,3284.0,49831.0,43.4,55956.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.377548,-88.056146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,Yale,430.0,200.0,230.0,409.0,18.0,70.0,0.0,0.0,0.0,0.0,430.0,55972.0,44.8,56292.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.119762,-88.024757
981,Yates City,1191.0,589.0,602.0,1186.0,32.0,20.0,0.0,0.0,0.0,0.0,1191.0,56607.0,45.2,87263.75,0.0,1.0,0.0,0.0,1.0,7.0,0.0,0.0,40.778649,-90.014837
982,Yorkville,24886.0,12460.0,12426.0,21503.0,1883.0,120.0,849.0,0.0,1193.0,3049.0,21837.0,95878.0,35.5,253435.08,0.0,8.0,3.0,7.0,17.0,178.0,5.0,2.0,41.641141,-88.447295
983,Zeigler,1794.0,915.0,879.0,1786.0,6.0,29.0,7.0,0.0,29.0,34.0,1760.0,43672.0,32.8,39465.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.899495,-89.052019


# Upload final_table to PGAdmin

In [23]:
#Convert final_table back into Spark
final_table = spark.createDataFrame(final_table)

In [24]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to final_table table in RDS
final_table.write.jdbc(url=jdbc_url, table='final_table', mode=mode, properties=config)